Source: https://github.com/eliasmelul/finding_schitts/blob/master/Recommender_System_FMSC.ipynb

In [6]:
%pip install pandas
%pip install scikit-learn
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\vishr\VSCodeProjects\CS 222\Project\api\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\vishr\VSCodeProjects\CS 222\Project\api\Scripts\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 7.6/7.6 MB 2.4 MB/s eta 0:00:00
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
     ---------------------------------------- 55.4/55.4 KB 2.8 MB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     -------------------------------------- 160.2/160.2 KB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 22.8 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 32.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\vishr\VSCodeProjects\CS 222\Project\api\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
norm_df = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/normalized_df_sub.csv', index_col=0)

In [199]:
cleaned_df = pd.read_csv('cleaned_city_temperatures.csv', index_col=0)

In [200]:
nan_cols = cleaned_df.columns[cleaned_df.isna().any()]
nan_cols

Index([], dtype='object')

In [201]:
def from_city_cosSim(data, name):
    try:
        Xs = data[data.City == name].drop(columns=['City'], axis=1)
        Col_A = data[data.City != name].City
        Ys = data[data.City != name].drop(columns=['City'], axis=1)

        cosSim = cosine_similarity(X=Xs,Y=Ys)
        sim = list(cosSim[0])
        cty = list(Col_A)
        comb = {"City":cty,"Similarity":sim}
        dfdf = pd.DataFrame(comb).reset_index()

        # Adding a row with the Y City
        currCity = {"City":name,"Similarity":1}
        curr = pd.DataFrame(currCity, index=[0])

        # Concatenate to finalize DF
        dfdf = pd.concat([dfdf,curr], sort=False).reset_index(drop=True).drop('index',1)
        return(dfdf)
    except:
        print("Wrong input: this entry will be ignored")

In [202]:
def input_cities(numShow=10):
    # create class that defines cities
    class rated_city:
        def __init__(self, city):
            self.city = city
    
    #Loop to input cities based on the user
    add_city = True
    userInput = []
    w = 0
    while add_city == True:
        city_name = input("City (Include state - Ex. New York, NY): ")
        userInput.append(city_name)
        simSim = from_city_cosSim(data=cleaned_df, name=city_name)
        try:
            cosSim = cosSim.merge(simSim, how='inner', on='City')
        except:
            cosSim = simSim
        
        city = rated_city(city_name)
        cont = input("Do you want to include another city?")
        add_city = cont.lower() in ['yes','true','of course','y','si','1']
        w+=1
    
    simCols = cosSim.drop("City",1)
    cits = cosSim.City
        
    for i, row in simCols.iterrows():
        simCols.at[i,'SumVal'] = row.sum()/w
    simi = simCols.SumVal
    
    out = {"City":cits,"Score":simi}
    out = pd.DataFrame(out).set_index("City").drop(userInput)
    out = out.sort_values('Score', ascending=False)
    
    return out.head(numShow)      



In [203]:
prediction = input_cities()

C:\Users\vishr\AppData\Local\Temp\ipykernel_42168\2615273898.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dfdf = pd.concat([dfdf,curr], sort=False).reset_index(drop=True).drop('index',1)
C:\Users\vishr\AppData\Local\Temp\ipykernel_42168\371256438.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  simCols = cosSim.drop("City",1)


In [204]:
prediction.to_dict('dict')

{'Score': {'Vancouver': 0.9987170950245319,
  'Salem': 0.9984088144949581,
  'Eugene': 0.9979385270146347,
  'Medford': 0.9956179749045557,
  'Fresno': 0.9955112146760742,
  'Sacramento': 0.9955110478881054,
  'Beirut': 0.995345851528191,
  'Portland': 0.9951277896464226,
  'Manama': 0.9947578597861054,
  'Cairo': 0.9946456269327455}}

In [188]:
df = pd.read_csv('./city_temperature.csv')

C:\Users\vishr\AppData\Local\Temp\ipykernel_42168\4205948694.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./city_temperature.csv')


In [189]:
df = df[df['Year'] == 2012]

In [190]:
df['Date'] = df.apply(lambda x: str(x['Month']) + '-' + str(x['Day']) + '-' + str(x['Year']), axis=1)

In [191]:
pivoted_df = df.pivot_table(index='City', columns='Date', values='AvgTemperature')

In [192]:
pivoted_df

Date,1-1-2012,1-10-2012,1-11-2012,1-12-2012,1-13-2012,1-14-2012,1-15-2012,1-16-2012,1-17-2012,1-18-2012,...,9-28-2012,9-29-2012,9-3-2012,9-30-2012,9-4-2012,9-5-2012,9-6-2012,9-7-2012,9-8-2012,9-9-2012
City,,,,,,,,,,,,,,,,,,,,,
Abidjan,77.1,81.0,81.8,82.1,82.1,78.9,78.7,76.6,77.9,81.2,...,80.2,79.6,74.1,79.9,75.9,75.5,74.6,77.0,75.1,76.9
Abilene,48.2,41.5,46.9,37.3,38.0,45.9,51.4,61.5,51.5,33.6,...,69.1,65.6,86.7,67.3,89.4,90.3,87.0,90.8,75.6,72.5
Abu Dhabi,67.2,71.8,73.1,74.4,76.5,71.6,66.6,64.6,64.7,65.4,...,92.9,90.5,95.3,89.3,95.9,96.2,94.9,94.2,94.5,96.6
Addis Ababa,59.6,-99.0,-99.0,63.3,62.3,62.3,62.2,61.8,62.7,61.4,...,63.6,62.4,57.4,62.1,58.3,59.6,59.4,60.2,58.7,60.2
Akron Canton,40.7,35.5,35.1,42.3,22.1,18.4,18.7,26.5,46.7,32.0,...,54.6,53.6,72.4,55.6,74.0,73.2,75.9,73.7,65.8,60.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Winnipeg,23.0,34.1,14.6,0.9,-5.3,12.2,21.0,6.9,-11.5,-13.5,...,55.3,62.5,67.3,65.8,67.6,61.3,56.2,55.0,58.1,52.5
Yakima,34.6,37.1,28.7,20.6,22.0,23.7,25.6,21.6,32.1,31.6,...,59.0,63.5,70.7,64.1,68.4,66.9,72.9,65.7,67.1,72.6
Youngstown,40.6,36.3,33.8,41.7,24.0,19.0,16.8,26.1,46.7,31.7,...,54.9,51.0,70.0,52.2,72.8,70.2,73.5,70.7,64.5,58.6


In [193]:
pivoted_df.drop_duplicates(inplace=True)
pivoted_df = pivoted_df.reset_index()

In [194]:
pivoted_df = pivoted_df.drop(['1-1-2012', '12-31-2012', '2-29-2012', '3-0-2012'], axis=1)

In [195]:
pivoted_df

Date,City,1-10-2012,1-11-2012,1-12-2012,1-13-2012,1-14-2012,1-15-2012,1-16-2012,1-17-2012,1-18-2012,...,9-28-2012,9-29-2012,9-3-2012,9-30-2012,9-4-2012,9-5-2012,9-6-2012,9-7-2012,9-8-2012,9-9-2012
0,Abidjan,81.0,81.8,82.1,82.1,78.9,78.7,76.6,77.9,81.2,...,80.2,79.6,74.1,79.9,75.9,75.5,74.6,77.0,75.1,76.9
1,Abilene,41.5,46.9,37.3,38.0,45.9,51.4,61.5,51.5,33.6,...,69.1,65.6,86.7,67.3,89.4,90.3,87.0,90.8,75.6,72.5
2,Abu Dhabi,71.8,73.1,74.4,76.5,71.6,66.6,64.6,64.7,65.4,...,92.9,90.5,95.3,89.3,95.9,96.2,94.9,94.2,94.5,96.6
3,Addis Ababa,-99.0,-99.0,63.3,62.3,62.3,62.2,61.8,62.7,61.4,...,63.6,62.4,57.4,62.1,58.3,59.6,59.4,60.2,58.7,60.2
4,Akron Canton,35.5,35.1,42.3,22.1,18.4,18.7,26.5,46.7,32.0,...,54.6,53.6,72.4,55.6,74.0,73.2,75.9,73.7,65.8,60.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,Winnipeg,34.1,14.6,0.9,-5.3,12.2,21.0,6.9,-11.5,-13.5,...,55.3,62.5,67.3,65.8,67.6,61.3,56.2,55.0,58.1,52.5
295,Yakima,37.1,28.7,20.6,22.0,23.7,25.6,21.6,32.1,31.6,...,59.0,63.5,70.7,64.1,68.4,66.9,72.9,65.7,67.1,72.6
296,Youngstown,36.3,33.8,41.7,24.0,19.0,16.8,26.1,46.7,31.7,...,54.9,51.0,70.0,52.2,72.8,70.2,73.5,70.7,64.5,58.6
297,Zagreb,41.8,35.6,34.4,40.0,36.7,30.7,28.6,33.4,35.2,...,60.7,64.9,75.0,63.9,75.2,73.9,71.9,66.5,68.2,72.3


In [198]:
pivoted_df.to_csv('./cleaned_city_temperatures.csv')